#İkinci El Araç Tahmin uYGULAMASI

In [190]:
import os
import numpy as np
import pandas as pd

In [191]:
df = pd.read_csv("kudet_arac_degerleme/data/arabamcom_merged.csv")

In [192]:
print(df.columns.tolist())

['smallest-text-minus href', 'listing-image src', 'listing-text-new', 'listing-text-new 2', 'fade-out-content-wrapper', 'fade-out-content-wrapper href 2', 'db', 'fade-out-content-wrapper 2', 'fade-out-content-wrapper 3', 'fade-out-content-wrapper 4']


In [193]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   smallest-text-minus href         1565 non-null   object
 1   listing-image src                1565 non-null   object
 2   listing-text-new                 1565 non-null   object
 3   listing-text-new 2               1565 non-null   object
 4   fade-out-content-wrapper         1565 non-null   int64 
 5   fade-out-content-wrapper href 2  1441 non-null   object
 6   db                               1565 non-null   object
 7   fade-out-content-wrapper 2       1180 non-null   object
 8   fade-out-content-wrapper 3       1562 non-null   object
 9   fade-out-content-wrapper 4       1562 non-null   object
dtypes: int64(1), object(9)
memory usage: 122.4+ KB
None


In [194]:
print(df.columns)

Index(['smallest-text-minus href', 'listing-image src', 'listing-text-new',
       'listing-text-new 2', 'fade-out-content-wrapper',
       'fade-out-content-wrapper href 2', 'db', 'fade-out-content-wrapper 2',
       'fade-out-content-wrapper 3', 'fade-out-content-wrapper 4'],
      dtype='object')


In [198]:
# Gereksiz sütunları sil
df.drop(columns=[
    'listing-image src',
    'fade-out-content-wrapper href 2',
    'fade-out-content-wrapper 2'
], inplace=True, errors='ignore')

# Sütunları yeniden adlandır
df.rename(columns={
    'smallest-text-minus href': 'ilan_linki',
    'listing-text-new': 'model',
    'listing-text-new 2': 'model_detay',
    'fade-out-content-wrapper': 'model_yili',
    'db': 'fiyat',
    'fade-out-content-wrapper 3': 'sehir',
    'fade-out-content-wrapper 4': 'ilce'
}, inplace=True)


In [199]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ilan_linki   1565 non-null   object
 1   model        1565 non-null   object
 2   model_detay  1565 non-null   object
 3   model_yili   1565 non-null   int64 
 4   fiyat        1565 non-null   object
 5   sehir        1562 non-null   object
 6   ilce         1562 non-null   object
dtypes: int64(1), object(6)
memory usage: 85.7+ KB
None


In [200]:
print(df["fiyat"].unique()[:10])  # ilk 10 farklı değeri göster

['1.350.000 TL' '1.025.000 TL' '1.910.900 TL' '1.119.900 TL'
 '1.179.900 TL' '439.000 TL' '2.100.000 TL' '2.250.000 TL' '1.850.000 TL'
 '2.799.000 TL']


In [201]:
# TL simgelerini ve boşlukları kaldır, sonra sayıya çevir
df["fiyat"] = df["fiyat"].astype(str).str.replace("[^0-9]", "", regex=True)
df["fiyat"] = pd.to_numeric(df["fiyat"], errors="coerce")

In [202]:
print(df["fiyat"].head(10))
print(df["fiyat"].dtype)
print(df["fiyat"].isnull().sum())  # kaç tanesi NaN olmuş

0    1350000
1    1025000
2    1910900
3    1119900
4    1179900
5     439000
6    2100000
7    2250000
8    1850000
9    2799000
Name: fiyat, dtype: int64
int64
0


In [206]:
def parse_ilan_link(link):
    try:
        if not isinstance(link, str):
            return {}

        parts = link.split("/")
        if len(parts) < 5:
            return {}

        segment = parts[4]
        pieces = segment.split("-")

        return {
            "ilan_turu": pieces[0] if len(pieces) > 0 else None,
            "tip": pieces[1] if len(pieces) > 1 else None,
            "marka": pieces[2] if len(pieces) > 2 else None,
            "model": pieces[3] if len(pieces) > 3 else None,
            "motor_hacmi": ".".join(pieces[4:6]) if len(pieces) > 5 else None,
            "motor_tipi": pieces[6] if len(pieces) > 6 else None,
            "donanim": pieces[7] if len(pieces) > 7 else None,
            "donanim_detay": pieces[8] if len(pieces) > 8 else None,
            "model_yili": pieces[9] if len(pieces) > 9 else None,
            "yakit": pieces[10] if len(pieces) > 10 else None,
            "vites": pieces[11] if len(pieces) > 11 else None
        }
    except Exception as e:
        print("Hata:", e)
        return {}


In [207]:
parsed_df = df["ilan_linki"].apply(parse_ilan_link).apply(pd.Series)
df = pd.concat([df, parsed_df], axis=1)
df = df.loc[:, ~df.columns.duplicated()]



In [208]:
print(df[["marka", "model", "motor_hacmi", "vites"]].sample(5))


          marka                                             model motor_hacmi  \
465     peugeot                  Peugeot 308 1.6 THP Feeline Plus         1.6   
215     renault                           Renault Captur 1.2 Icon         1.2   
709  volkswagen  Volkswagen Passat 1.4 TSi BlueMotion Comfortline         1.4   
726     citroen                         Citroen C3 1.4 Attraction         1.4   
866        fiat                     Fiat Doblo Cargo 1.6 Multijet     cargo.1   

    vites  
465  None  
215  None  
709  None  
726  None  
866  None  


In [209]:
print(df.columns)

Index(['ilan_linki', 'model', 'model_detay', 'model_yili', 'fiyat', 'sehir',
       'ilce', 'ilan_turu', 'tip', 'marka', 'motor_hacmi', 'motor_tipi',
       'donanim', 'donanim_detay', 'yakit', 'vites'],
      dtype='object')


In [210]:
print(df['marka'].unique())
print(df['model_yili'].unique())
print(df['vites']
)

['jeep' 'hyundai' 'chery' 'fiat' 'renault' 'mercedes' 'volvo' 'nissan'
 'skoda' 'peugeot' 'seat' 'audi' 'bmw' 'volkswagen' 'toyota' 'honda'
 'citroen' 'ford' 'tofas' 'opel' 'ssangyong' 'land' 'kia' 'dacia' 'iveco'
 'porsche' 'mitsubishi' 'yamaha' 'river' 'jaguar' 'kuba' 'chevrolet'
 'maserati' 'ticari' 'bajaj' 'mini' 'tvs' 'baoli' 'togg' 'karsan' 'kral'
 'skywell' 'alfa' 'car' 'mazda' 'mg' 'revolt' 'isuzu' 'smart' 'ds' 'new'
 'universal' 'yuki' 'suzuki' 'otokar' 'yiben' 'lamborghini']
[2019 2013 2024 2022 2017 2012 2021 1997 2011 2014 2015 2025 2023 2016
 2018 2003 2020 2010 1993 2009 2007 2000 2005 2006 1988 1998 1999 1992
 2004 2008 1994 2001 1991 1996 1990 2002 1995]
0       None
1       None
2       None
3       None
4       None
        ... 
1560    None
1561    None
1562    None
1563    None
1564    None
Name: vites, Length: 1565, dtype: object


In [211]:
print(df.columns.tolist())

['ilan_linki', 'model', 'model_detay', 'model_yili', 'fiyat', 'sehir', 'ilce', 'ilan_turu', 'tip', 'marka', 'motor_hacmi', 'motor_tipi', 'donanim', 'donanim_detay', 'yakit', 'vites']


In [212]:
print(df.isnull().sum())


ilan_linki          0
model               0
model_detay         0
model_yili          0
fiyat               0
sehir               3
ilce                3
ilan_turu           0
tip                 0
marka               0
motor_hacmi        68
motor_tipi        105
donanim           354
donanim_detay    1008
yakit            1556
vites            1561
dtype: int64


In [213]:
df.drop(columns=["vites", "yakit", "donanim_detay"], inplace=True)


In [214]:
df["motor_hacmi"].fillna("bilinmiyor", inplace=True)
df["motor_tipi"].fillna("bilinmiyor", inplace=True)
df["donanim"].fillna("bilinmiyor", inplace=True)


In [215]:
print(df.isnull().sum())
print(df.shape)


ilan_linki     0
model          0
model_detay    0
model_yili     0
fiyat          0
sehir          3
ilce           3
ilan_turu      0
tip            0
marka          0
motor_hacmi    0
motor_tipi     0
donanim        0
dtype: int64
(1565, 13)


In [216]:
df = df[(df["model_yili"] >= 1980) & (df["fiyat"] >= 100000)]
print(df.shape)
print(df[["model_yili", "fiyat"]].describe())


(1551, 13)
        model_yili         fiyat
count  1551.000000  1.551000e+03
mean   2014.214055  1.117144e+06
std       7.178504  2.815328e+06
min    1988.000000  1.050000e+05
25%    2011.000000  5.342500e+05
50%    2015.000000  8.050000e+05
75%    2020.000000  1.227500e+06
max    2025.000000  6.200000e+07


In [217]:
print(df.columns)

Index(['ilan_linki', 'model', 'model_detay', 'model_yili', 'fiyat', 'sehir',
       'ilce', 'ilan_turu', 'tip', 'marka', 'motor_hacmi', 'motor_tipi',
       'donanim'],
      dtype='object')


In [218]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1551 entries, 0 to 1564
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ilan_linki   1551 non-null   object
 1   model        1551 non-null   object
 2   model_detay  1551 non-null   object
 3   model_yili   1551 non-null   int64 
 4   fiyat        1551 non-null   int64 
 5   sehir        1548 non-null   object
 6   ilce         1548 non-null   object
 7   ilan_turu    1551 non-null   object
 8   tip          1551 non-null   object
 9   marka        1551 non-null   object
 10  motor_hacmi  1551 non-null   object
 11  motor_tipi   1551 non-null   object
 12  donanim      1551 non-null   object
dtypes: int64(2), object(11)
memory usage: 169.6+ KB
None


In [219]:
df.drop(columns=["ilan_linki"], inplace=True)


In [220]:
print(df.columns)

Index(['model', 'model_detay', 'model_yili', 'fiyat', 'sehir', 'ilce',
       'ilan_turu', 'tip', 'marka', 'motor_hacmi', 'motor_tipi', 'donanim'],
      dtype='object')


In [221]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1551 entries, 0 to 1564
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   model        1551 non-null   object
 1   model_detay  1551 non-null   object
 2   model_yili   1551 non-null   int64 
 3   fiyat        1551 non-null   int64 
 4   sehir        1548 non-null   object
 5   ilce         1548 non-null   object
 6   ilan_turu    1551 non-null   object
 7   tip          1551 non-null   object
 8   marka        1551 non-null   object
 9   motor_hacmi  1551 non-null   object
 10  motor_tipi   1551 non-null   object
 11  donanim      1551 non-null   object
dtypes: int64(2), object(10)
memory usage: 157.5+ KB
None


In [222]:
# En sık geçen şehir ve ilçe değerlerini al
sehir_mode = df["sehir"].mode()[0]
ilce_mode = df["ilce"].mode()[0]

# Eksik olanları bunlarla doldur
df["sehir"].fillna(sehir_mode, inplace=True)
df["ilce"].fillna(ilce_mode, inplace=True)


In [223]:
print(df[["sehir", "ilce"]].isnull().sum())

sehir    0
ilce     0
dtype: int64


In [226]:
# Benzersiz motor hacmi değerlerini al, boş olmayanları sırala
print(df["motor_hacmi"].dropna().unique())

['1.6' '8.pro' 'cross.1' '1.5' 'axor.1840' 'sx.2' 'e.180' '1.0' '2.0'
 'serisi.520i' '3.0' 'serisi.418i' '1.4' 'bilinmiyor' 'combi.1' '2.4'
 'variant.1' '1.8' 'elysee.1' 'sedan.1' '1.3' 'sedan.35' 'courier.1'
 '19.1' 'aircross.1' 'sportback.45' 'serisi.216d' 'city.1' 'e.200' 'e.300'
 'freelander.2' 'express.1' '3.2' '1.2' 'blue.1' '1.33' 'x.1'
 'serisi.316i' 'trail.1' 'connect.1' 'hr.1' '2.3' '72.160' 'gt3.rs'
 '300.city' '0.9' 'a.180' 'sportback.1' 'c.180' 'serisi.320i' 'cargo.2520'
 '9.1' 'evo.1' 'sedan.2' 'cargo.1' 'c.200' 'cla.200' '16d.sdrive' '2.2'
 'roc.1' '2.7' 'sportback.2' 'up.4wd' 'v.2' 'range.evoque' 'serisi.520d'
 'serisi.730d' '2.5' 'serisi.m5' 'canter.8bl' 'dorse.kuru' '1200.x'
 '200.rs' '300.s' 'transit.350' 'eqb.250' 'e.220' 'transit.14'
 'transit.16' 'panorama.1' 'countryman.1' 's.350' '1.9' 'transit.t330'
 '1.6d' '1.25' 'glk.220' '15.1' 'kr.305' '13.m3' 'serisi.320d' 'glc.350'
 'cherokee.5' 'serisi.318d' 'premium.plus' 'serisi.116i' 'era.1'
 'multix.1' 'tonale.1' 'a.

In [227]:
# En çok geçen motor hacmi değerlerini frekansla birlikte göster
print(df["motor_hacmi"].value_counts(dropna=False).head(30))  # ilk 30 değer

motor_hacmi
1.6            302
1.5            198
1.4            150
1.3            106
1.2             89
combi.1         61
bilinmiyor      55
2.0             50
1.0             48
cross.1         21
courier.1       17
elysee.1        16
connect.1       16
sedan.2         15
sedan.1         13
serisi.320i     13
cargo.1         13
1.9             12
1.8             12
serisi.520i     11
c.180           11
a.180           10
e.220           10
3.0              9
s.350            9
aircross.1       9
blue.1           8
panorama.1       8
serisi.316i      7
0.9              7
Name: count, dtype: int64


In [224]:
pd.set_option("display.max_columns", None)  # Tüm sütunlar görünsün
print(df.sample(50))


                                                 model  \
889                  Renault Megane 1.5 dCi Touch Plus   
1186                          Seat Arona 1.0 EcoTSI FR   
1496             Ticari Araçlar Dorse Kuru Yük Kapaklı   
956                      Renault Fluence 1.5 dCi Touch   
1026            Citroen C3 Aircross 1.2 PureTech Shine   
954                      Fiat Egea 1.6 Multijet Lounge   
110                 Nissan Qashqai 1.5 dCi Design Pack   
607                  Renault Fluence 1.5 dCi Privilege   
1343                 Audi A3 Sedan 1.4 TFSI Attraction   
851              Volkswagen Tiguan 1.4 TSI Sport&Style   
904                      Skoda SuperB 2.0 TDI Prestige   
941                     Opel Crossland 1.2 T Essential   
178                 Fiat Doblo Combi 1.6 Multijet Easy   
932   Volkswagen Passat 1.6 TDi BlueMotion Comfortline   
377                      Skoda SuperB 2.0 TDI Prestige   
552                Fiat Fiorino Combi 1.3 Multijet Pop   
756           

In [225]:
df.to_csv("temizlenmis_arac_verisi.csv", index=False)